## Comprovació absència NA i Outliers df

In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit
import duckdb

In [3]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/16 15:58:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [102]:
# rent_price
DF = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/rent_price.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT * FROM RDD") \
  .load()

# verificar duplicats
print(f"Hi ha {DF.count() - DF.distinct().count()} valors duplicats al DataFrame 'formatted_zone/rent_price.db' de {DF.count()} valors totals")

# verificar NA
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs')

# escalar obteninint mínim i màxims via query en funció de la unitat
min_max = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/rent_price.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", 'SELECT "Average _rent", min(Price) AS min_price , max(Price) AS max_price FROM RDD GROUP BY "Average _rent"') \
  .load().collect()

DF = DF.withColumn('Price', F.when(DF['Average _rent'] == min_max[0][0], (DF['Price'] - min_max[0].min_price)/(min_max[0].max_price - min_max[0].min_price)).otherwise((DF['Price'] - min_max[1].min_price)/(min_max[1].max_price - min_max[1].min_price)))

print("S'ha escalat la columna 'Price' en funció de la columna 'Average _rent'")

# outliers
IQR = DF.approxQuantile('Price', [0.25, 0.75], 0.001)
IQR_values = IQR[1] - IQR[0]
        
DF = DF.withColumn('IQR', lit(IQR_values))
                
not_outliers = DF.filter(col('Price') >= IQR[0] - 1.5 * IQR_values)\
            .filter(col('Price') <= IQR[1] + 1.5 * IQR_values)

outliers = DF.subtract(not_outliers)
print(f"Hi ha {outliers.count()} outliers detectats per cuartils")

DF = DF.drop('IQR')

Hi ha 0 valors duplicats al DataFrame 'formatted_zone/rent_price.db' de 4622 valors totals
Hi ha 0 NAs
S'ha escalat la columna 'Price' en funció de la columna 'Average _rent'
Hi ha 36 outliers detectats per cuartils
